In [1]:
import numpy as np
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import datetime
import time

class Scrape():
    """
    Parent class for bookmaker classes.
    Contains the methods that are common for all bookmakers.


    """

    def __init__(self):
        """
        Initializes class Scape with empty dataframe for storing data of a bookmaker.

        Two variables created: self.df, self.link

        self.link - the link of the website which directs to the certain league page
        self.df - empty dataframe for storing data of the bookmaker
        """

        self.df = pd.DataFrame(columns = ['date','day','time','title','1','X','2'])

        self.link = None


    def get_html(self):
        """
        Runs a chrome driver for loading all the necessary parts of a website.
        BeautifulSoup library gets the html of the page and parses for getting a usable information.
        The driver closes at the end of the operations

        Variables created: self.soup - contains the html code of a certain league

        """

        self.driver = webdriver.Chrome(ChromeDriverManager().install())

        url = self.link

        self.driver.get(url)

        time.sleep(30) #Wait 30 seconds for the page to load   

        page_source = self.driver.page_source

        print("Browser loaded")

        self.soup = BeautifulSoup(page_source, 'html.parser')

        print("Done")

        self.driver.close()

class Vivaro(Scrape):
    """
    Vivaro bookmaker class, scrapes the content of Vivaro website of a certain league, stores the values in a dataframe.
    """

    def __init__(self):
        """
        Initializes the class throught parent class, which creates an empty dataframe for storing data.

        Global variables created: self.link, self.df

        self.link - the link of the website which directs to the certain league page
        self.df - empty dataframe for storing data of the bookmaker

        Local variables created: None

        """

        super().__init__() 

        self.link = "https://www.vivarobet.am/#/sport/?type=0&region=10075&competition=1855&sport=1&game=16949220"

    def get_table(self):
        """
        Gets table of values of a certain league

        Global variables created: self.dataframe

        self.dataframe - the table of the coefficients

        Local variables created: td

        td - variable for getting tables with certain class from html code

        """

        td = self.soup.find_all('table', class_ = "aic-hdp-row")

        self.dataframe = pd.read_html(str(td))

    def get_games(self):

        """
        Stores all the data into a dataframe.
        The result is a dataframe with columns title, day, time, date, hometeam win, awayteam win and draw.

        """

        self.dataframe = self.dataframe[0::3] 

        dayCodeList = self.soup.find_all('div', class_ = 'multicolumn-table') # Get separate days code with their games

        gamesList = self.soup.find_all('table', class_ = 'aic-hdp-row') # Get the code of all the games

        week_days = ["Երկուշաբթի","Երեքշաբթի","Չորեքշաբթի","Հինգշաբթի","Ուրբաթ","Շաբաթ","Կիրակի"]

        dating = {'01' : 'Հունվ', '02' : 'Փետր', '03' : 'Մարտ', '04': 'Ապր', '05': 'Մայ', '06' : 'Հուն', '07' : 'Հուլ', '08' : 'Օգոստոս', '09' : 'Սեպտ', '10' : 'Հոկտ','11' : 'Նոյ', '12' : 'Դեկտ'}

        eachGameDate = {}

        try:

            for gameIndex in range(len(gamesList)): # for games in gameslist

                for dayIndex in range(len(dayCodeList)): # for day in daylist

                    if gamesList[gameIndex] in dayCodeList[dayIndex]: # if game in day

                        eachGameDate["game{0}".format(gameIndex)] = dayCodeList[dayIndex].find('div', class_ = 'time-title-view-v3').get_text() # make game date equal to the day date

        except:

            print("Error")


        self.df = pd.concat(self.dataframe) 

        self.df = self.df.loc[::4, :]

        self.df.rename(columns={0: 'title', 1: '1', 2 : 'X', 3 : '2'}, inplace = True)

        self.df.insert(0, 'time', 0), self.df.insert(0, 'day', 0), self.df.insert(0, 'date', 0)

        try:

            self.df['time'] = self.df['title'].apply(lambda x: re.search(r"[0-9]+:[0-9]+", x).group())

        except:

            print("Error while taking the time of one of the games")

        try:

            self.df['title'] = self.df['title'].apply(lambda x: '%s vs %s' % (tuple(re.split("\d+:\d+soon", x))))

        except:

            print("Error while taking the title of one of the games")

        self.df.reset_index(drop=True, inplace = True)

        for index in range(len(eachGameDate)):

            self.date_regex = eachGameDate["game{0}".format(index)].split('.') 

            try:

                self.df['date'].loc[index] = self.date_regex[0] + ' ' + dating[self.date_regex[1]]   

            except:

                print("Error while taking the date of one of the games")

            try:

                weekNum = datetime.date(int(self.date_regex[2]),int(self.date_regex[1][-1]),int(self.date_regex[0])).weekday()

            except:

                print("Error while taking the day of one fo the games")

            try:

                self.df['day'].loc[index] = week_days[weekNum]        

            except:

                print("Error while taking the day of one of the games")

        self.df[['1', '2', 'X']] = self.df[['1', '2', 'X']].apply(pd.to_numeric)

        self.df = self.df.iloc[:,0:7]

        print(self.df)




class Adjara(Scrape):

    """
    Adjarabet bookmaker class, scrapes the content of Adjarabet of a certain league, stores the values in a dataframe.

    """

    def __init__(self):

        """
        Initializes the class throught parent class, which creates an empty dataframe for storing data.

        Global variables created: self.link, self.df

        self.link - the link of the website which directs to the certain league page
        self.df - empty dataframe for storing data of the bookmaker

        Local variables created: None

        """
        super().__init__()

        self.link = "https://sportsbook.adjarabet.am/home/wrapper/events?lang=hy&leagues=%5B2449%5D"

    def get_games(self):

        """
        Gets separate game html code and stores into a dictionary.

        Global variables created: 

        self.dic
            A dictionary: stores separate game data. (Ex. "game1": "separate game html code")
        self.day_of_game
            A dictionary: stores separate game date. (Ex. "game1": "separate game date")


        """

        self.dic = {}

        #Creating empty dictionary for storing separate game's date and day

        self.day_of_game = {}

        #Storing data
        index = 1

        for day in self.soup.find_all('div', {"class" : "ng-star-inserted", "type" : "event-list"}):

            for game in day.find_all('div', {"class" : "sb-event-list__row sb-event-list__row--desktop ng-star-inserted"}):

                try:

                    self.dic["game{0}".format(index)] = game

                    self.day_of_game["game{0}".format(index)] = [day.find('span', {"class" : "sb-event-list__outcome-header__date-item sb-event-list__outcome-header__date-item--desktop ng-star-inserted"}).text, day.find('span', {"class" : "sb-event-list__outcome-header__date-week sb-event-list__outcome-header__date-week--desktop ng-star-inserted"}).text]
                    index+=1

                except:

                    print("Error while getting the information of " + str(index)+ " (st/nd/th) index")

    def store_values_pd(self):
        """
        Stores all the data into a dataframe.
        The result is a dataframe with columns title, day, time, date, hometeam win, awayteam win and draw.

        """

        for i in range(len(self.dic)):

            game_code = self.dic["game{0}".format(i+1)]

            try:

                self.df.loc[i, 'time'] = game_code.find("span", class_ = "sb-event-list__event__time").text

            except:

                print("Error while taking the time of one of the games")

            try:

                self.df.loc[i, 'day'] = self.day_of_game["game{0}".format(i+1)][1]

            except:

                print("Error while taking the day of one of the games")

            try:

                self.df.loc[i, 'date'] = self.day_of_game["game{0}".format(i+1)][0]

            except:

                print("Error while taking the date of one of the games")

            try:

                self.df.loc[i, 'title'] = "%s vs %s" % (game_code.find_all("span", class_ = "sb-event-list__competitor sb-event-list__competitor--prematch")[0].get_text().strip(),game_code.find_all("span", class_ = "sb-event-list__competitor sb-event-list__competitor--prematch")[1].get_text().strip())

            except:

                print("Error while taking the title of one of the games")

            try:

                coef = [k.get_text() for k in self.dic["game{0}".format(i+1)].find_all("span",{"class":"ng-star-inserted","pt":"","pt-item":"odd"})]

                self.df.loc[i, '1'] = coef[0]
                self.df.loc[i, 'X'] = coef[1]
                self.df.loc[i, '2'] = coef[2]

            except:

                print("Error while getting the coefficients of one of the games")
        try:

            self.df[['1', '2', 'X']] = self.df[['1', '2', 'X']].apply(pd.to_numeric)
        except:

            print("Couldn't make columns '1','2','X' numeric")



    def start_prog(self):

        self.get_html()
        self.get_games()
        self.store_values_pd()






class Toto(Scrape):
    """
    TotoGaming bookmaker class, scrapes the content of TotoGaming of a certain league, stores the values in a dataframe.

    """

    def __init__(self):
        """
        Initializes the class throught parent class, which creates an empty dataframe for storing data.

        Global variables created: self.link, self.df

        self.link - the link of the website which directs to the certain league page
        self.df - empty dataframe for storing data of the bookmaker

        Local variables created: None

        """
        super().__init__()
        self.link = "https://sport.totogaming.am/SportsBook/Upcoming/27008/?game=%D4%B5%D5%BE%D6%80%D5%B8-2020&gameId=6465838" 



    def get_games(self):

        """
        Gets separate game html code and stores into a dictionary.

        Global variables created: 

        self.gamesCodeDict
            A dictionary: stores separate game data. (Ex. "game1": "separate game data")


        """

        self.get_html()

        gamesList = self.soup.find_all('div', {'class' :['tg__match_item tg_widget_bg tg--mar-b-16', 'tg__match_item tg_widget_bg tg--mar-b-16 tg--selected_prem_game']})

        self.gamesCodeDict = {}

        for index in range(len(gamesList)):

            self.gamesCodeDict["game{0}".format(index)] = gamesList[index] 

        return self.gamesCodeDict

    def store_values(self):
        """
        Stores all the data into a dataframe.
        The result is a dataframe with columns title, day, time, date, hometeam win, awayteam win and draw.

        """

        week_days = ["Երկուշաբթի","Երեքշաբթի","Չորեքշաբթի","Հինգշաբթի","Ուրբաթ","Շաբաթ","Կիրակի"]

        dating = {'01' : 'Հունվ', '02' : 'Փետր', '03' : 'Մարտ', '04': 'Ապր', '05': 'Մայ', '06' : 'Հուն', '07' : 'Հուլ', '08' : 'Օգոստոս', '09' : 'Սեպտ', '10' : 'Հոկտ','11' : 'Նոյ', '12' : 'Դեկտ'}

        for i in range(len(self.gamesCodeDict)):

            gameOtherInfo = [item.get_text() for item in self.gamesCodeDict["game{0}".format(i)].find_all('div', class_ = 'tg--mar-r-8')]


            self.df.loc[i, 'title'] = " vs ".join([item.get_text() for item in self.gamesCodeDict["game{0}".format(i)].find_all('div', class_ = 'en prematch_name tg--oe')])
            self.df.loc[i, '1'], self.df.loc[i, 'X'], self.df.loc[i, '2'] = tuple([item.get_text() for item in self.gamesCodeDict["game{0}".format(i)].find_all('div', class_ = 'prematch_stake_odd_factor l_od Decimal')])
            self.df.loc[i, 'time'] = gameOtherInfo[2]


            date = (gameOtherInfo[1]+"-21").split("-")

            week_num = datetime.date(int(date[2]),int(date[1][-1]),int(date[0])).weekday()

            self.df.loc[i, 'day'] = week_days[week_num]
            self.df.loc[i, 'date'] = "%s %s" % (date[0], dating[date[1]])

        self.df[['1', '2', 'X']] = self.df[['1', '2', 'X']].apply(pd.to_numeric)

        return self.df




class GoodW(Scrape):
    """
    TotoGaming bookmaker class, scrapes the content of TotoGaming of a certain league, stores the values in a dataframe.

    """

    def __init__(self):
        """
        Initializes the class throught parent class, which creates an empty dataframe for storing data.

        Global variables created: self.link, self.df

        self.link - the link of the website which directs to the certain league page
        self.df - empty dataframe for storing data of the bookmaker

        Local variables created: None

        """

        super().__init__()
        self.link = "https://sport.goodwin.am/am/sport/tournament/18528" 


    def get_games(self):
        """
        Gets separate game html code and stores into a dictionary.

        Global variables created: 

        self.gamesCodeDict
            A dictionary: stores separate game data. (Ex. "game1": "separate game data")


        """

        self.get_html()

        gamesList = self.soup.find_all('div', {'class' : ['marginTop___1WoyI']})

        self.gamesCodeDict = {}

        for index in range(len(gamesList)):

            self.gamesCodeDict["game{0}".format(index)] = gamesList[index] 

        return self.gamesCodeDict

    def get_html(self):
        """

        Overwritten method of parent class, which uses scrolling for loading all the data

        Runs a chrome driver for loading all the necessary parts of a website.
        BeautifulSoup library gets the html of the page and parses for getting a usable information.
        The driver closes at the end of the operations

        Variables created: self.soup - contains the html code of a certain league

        """

        #Getting html code of a separate league

        driver = webdriver.Chrome(ChromeDriverManager().install())
        url = self.link
        driver.get(url)
        time.sleep(10)
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            try:
                show_more = driver.find_element_by_xpath('//section[@name="sport1"]/div[1]/div[1]/button[1]')
                show_more.click()
            except:

                print("Scrolled")

            # Wait to load page
            time.sleep(2)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")

            if new_height == last_height:
                break
            last_height = new_height


        page_source = driver.page_source
        print("Browser loaded")
        self.soup = BeautifulSoup(page_source, 'html.parser')

        print("Done")  

        driver.close()

    def store_values(self):
        """
        Stores all the data into a dataframe.
        The result is a dataframe with columns title, day, time, date, hometeam win, awayteam win and draw.

        """

        week_days = ["Երկուշաբթի","Երեքշաբթի","Չորեքշաբթի","Հինգշաբթի","Ուրբաթ","Շաբաթ","Կիրակի"]

        dating = {'Jan' : '01', 'Feb' : '02', 'Mar' : '03', 'Apr': '04', 'May': '05', 'Jun' : '06', 'Jul' : '07', 'Aug' : '08', 'Sep' : '09', 'Oct' : '10','Nov' : '11', 'Dec' : '12'}

        for i in range(len(self.gamesCodeDict)):

            self.df.loc[i, '1'], self.df.loc[i, 'X'], self.df.loc[i, '2'] = tuple([item.get_text() for item in self.gamesCodeDict["game{0}".format(i)].find_all('div', class_ = 'outcomeContent___1WRV5')][:3])
            self.df.loc[i, 'time'] = self.gamesCodeDict["game{0}".format(i)].find('div', class_ = 'time___LWRm_ margin___1KgHl').text
            self.df.loc[i, 'date'] = self.gamesCodeDict["game{0}".format(i)].find('div', class_ = 'date___1Injw').text
            self.df.loc[i, 'title'] = " vs ".join([item.get_text() for item in self.gamesCodeDict["game{0}".format(i)].find_all('div', class_ = 'name___wMGNI')])

            date = (self.df.loc[i, 'date']+" 21").split(" ")

            week_num = datetime.date(int(date[2]),int(dating[date[1]][-1]),int(date[0])).weekday()

            self.df.loc[i, 'day'] = week_days[week_num]

        self.df[['1', '2', 'X']] = self.df[['1', '2', 'X']].apply(pd.to_numeric)    

        return self.df









         

     

         
       





In [4]:
vivaro = Vivaro()
vivaro.get_html()
vivaro.get_table()
vivaro.get_games()     



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


Browser loaded
Done
       date         day   time                                  title      1  \
0   13 Հուն      Կիրակի  20:00       Ավստրիա vs Հյուսիսային Մակեդոնիա   1.79   
1   13 Հուն      Կիրակի  23:00               Նիդերլանդներ vs Ուկրաինա   1.57   
2   14 Հուն  Երկուշաբթի  17:00                     Շոտլանդիա vs Չեխիա   3.02   
3   14 Հուն  Երկուշաբթի  20:00                   Լեհաստան vs Սլովակիա   1.84   
4   14 Հուն  Երկուշաբթի  23:00                      Իսպանիա vs Շվեդիա   1.39   
5   15 Հուն   Երեքշաբթի  20:00               Հունգարիա vs Պորտուգալիա   8.50   
6   15 Հուն   Երեքշաբթի  23:00                    Ֆրանսիա vs Գերմանիա   2.81   
7   16 Հուն  Չորեքշաբթի  17:00                 Ֆինլանդիա vs Ռուսաստան   5.50   
8   16 Հուն  Չորեքշաբթի  20:00                       Թուրքիա vs Ուելս   2.45   
9   16 Հուն  Չորեքշաբթի  23:00                    Իտալիա vs Շվեյցարիա   1.67   
10  17 Հուն   Հինգշաբթի  17:00      Ուկրաինա vs Հյուսիսային Մակեդոնիա   1.71   
11  17 Հուն   Հինգշա

c:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [5]:
adjarabet = Adjara()
adjarabet.start_prog()
adjarabet.df      



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


Browser loaded
Done


,date,day,time,title,1,X,2
0,13 Հուն,Կիրակի,20:00,Ավստրիա vs Հյուսիսային Մակեդոնիա,1.85,3.50,4.95
1,13 Հուն,Կիրակի,23:00,Նիդեռլանդներ vs Ուկրաինա,1.60,4.00,6.50
2,14 Հուն,Երկուշաբթի,17:00,Շոտլանդիա vs Չեխիա,2.95,3.00,2.85
3,14 Հուն,Երկուշաբթի,20:00,Լեհաստան vs Սլովակիա,1.85,3.35,4.95
4,14 Հուն,Երկուշաբթի,23:00,Իսպանիա vs Շվեդիա,1.45,4.50,8.50
5,15 Հուն,Երեքշաբթի,20:00,Հունգարիա vs Պորտուգալիա,7.30,4.40,1.50
6,15 Հուն,Երեքշաբթի,23:00,Ֆրանսիա vs Գերմանիա,2.75,3.20,2.85
7,16 Հուն,Չորեքշաբթի,17:00,Ֆինլանդիա vs Ռուսաստան,5.50,3.80,1.70
8,16 Հուն,Չորեքշաբթի,20:00,Թուրքիա vs Ուելս,2.40,3.20,3.40
9,16 Հուն,Չորեքշաբթի,23:00,Իտալիա vs Շվեյցարիա,1.65,3.80,6.30


In [6]:
totogaming = Toto()
totogaming.get_games()
totogaming.store_values()    



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


Browser loaded
Done


,date,day,time,title,1,X,2
0,13 Հուն,Կիրակի,20:00,Ավստրիա vs Հս. Մակեդոնիա,1.79,3.62,5.43
1,13 Հուն,Կիրակի,23:00,Նիդերլանդներ vs Ուկրաինա,1.57,4.23,6.85
2,14 Հուն,Երկուշաբթի,17:00,Շոտլանդիա vs Չեխիա,2.96,3.04,2.82
3,14 Հուն,Երկուշաբթի,20:00,Լեհաստան vs Սլովակիա,1.83,3.37,5.64
4,14 Հուն,Երկուշաբթի,23:00,Իսպանիա vs Շվեդիա,1.39,4.88,10.57
5,15 Հուն,Երեքշաբթի,20:00,Հունգարիա vs Պորտուգալիա,8.29,4.35,1.49
6,15 Հուն,Երեքշաբթի,23:00,Ֆրանսիա vs Գերմանիա,2.76,3.20,2.89
7,16 Հուն,Չորեքշաբթի,17:00,Ֆինլանդիա vs Ռուսաստան,5.66,3.84,1.71
8,16 Հուն,Չորեքշաբթի,20:00,Թուրքիա vs Ուելս,2.47,3.11,3.41
9,16 Հուն,Չորեքշաբթի,23:00,Իտալիա vs Շվեյցարիա,1.67,3.90,6.08


In [8]:

goodwin = GoodW()
goodwin.get_games()
goodwin.store_values()     



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


Scrolled
Browser loaded
Done


,date,day,time,title,1,X,2
0,13 Jun,Կիրակի,20:00,Austria vs North Macedonia,1.77,3.65,5.30
1,13 Jun,Կիրակի,23:00,Netherlands vs Ukraine,1.58,4.35,6.00
2,14 Jun,Երկուշաբթի,17:00,Scotland vs Czech Republic,2.95,3.00,2.83
3,14 Jun,Երկուշաբթի,20:00,Poland vs Slovakia,1.85,3.40,5.20
4,14 Jun,Երկուշաբթի,23:00,Spain vs Sweden,1.38,4.85,10.00
5,15 Jun,Երեքշաբթի,20:00,Hungary vs Portugal,8.00,4.40,1.48
6,15 Jun,Երեքշաբթի,23:00,France vs Germany,2.76,3.20,2.85
7,16 Jun,Չորեքշաբթի,17:00,Finland vs Russia,5.80,3.70,1.71
8,16 Jun,Չորեքշաբթի,20:00,Turkey vs Wales,2.41,3.15,3.40
9,16 Jun,Չորեքշաբթի,23:00,Italy vs Switzerland,1.66,3.85,6.00


In [15]:
class Calculations:
    """
    Does all the calculations connected with the bookmakers


    """

    def __init__(self, **bm_dict):
        """
        Creates a table to store the highest coefficients for all the bookmakers

        Global Variables created:

        self.bm_dict
            dictionary containing all the bookmakers' dataframes
        self.highest_coef
            dataframe storing the highest coefficients from all the bookmakers


        """

        self.bm_dict = bm_dict

        self.highest_coef = pd.DataFrame(columns = ['date','day','time','title','1','X','2', 'bm-1','bm-X', 'bm-2', 'qt-1', 'qt-X','qt-2'])



    def rename(self, element):

        """
        Renames all the club names into one certain, for the class to be sorted with titles.

        """

#         clubnames_dic = {'Վալյադոլիդ' : ['Վալյադոլիդ', 'Ռեալ Վալյադոլիդ', 'R. Valladolid', 'Valladolid'], 'Բետիս': ['Բետիս', 'Ռեալ Բետիս', 'Real Betis', 'Betis'], 'Սոսիեդադ':['Սոսիեդադ', 'Ռեալ Սոսիեդադ', 'Սոսյեդադ','Ռեալ Սոսյեդադ','Real Sociedad', 'Sociedad', 'R Sociedad'], 'Ատլետիկ Բիլբաո': ['Ատլետիկ Բիլբաո','Ատլետիկ Բ', 'Athletic Club', 'Athletic', 'Athletic B'], 'Ատլետիկո Մադրիդ': ['Ատլետիկո Մադրիդ', 'Ատլետիկո Մ', 'Atletico Madrid', 'Atletico M', 'Atletico'], 'Ռեալ Մադրիդ':['Ռեալ Մադրիդ', 'Ռեալ Մ', 'Real Madrid', 'Real M', 'Real Madrid CF'], 'Էլչե':['Elche CF', 'Elche', 'Էլչե'], 'Վիլյառեալ': ['Վիլյառեալ', 'Villarreal CF', 'Villarreal'],'Էյբար': ['Էյբար', 'Eibar', 'SD Eibar'],'Գրանադա': ['Գրանադա', 'Granada CF', 'Granada'], 'Լևանտե': ['Լևանտե', 'Levante UD', 'Levante'], 'ՈՒեսկա': ['ՈՒեսկա', 'Huesca'], 'Բարսելոնա': ['Բարսելոնա', 'FC Barcelona', 'Barcelona'], 'Խետաֆե': ['Խետաֆե', 'Getafe CF', 'Getafe'], 'Սելտա': ['Սելտա', 'Celta', 'RC Celta'], 'Սելտա': ['Սելտա', 'Celta', 'RC Celta'], 'Ալավես': ['Ալավես', 'D. Alaves', 'Alaves'], 'Կադիս': ['Կադիս', 'Cadiz'], 'Վալենսիա': ['Վալենսիա', 'Valencia'], 'Սևիլիա': ['Սևիլիա', 'Sevilla', 'Sevilla FC'], 'Օսասունա': ['Օսասունա', 'Osasuna', 'CA Osasuna'], 'Բերնլի': ['Բերնլի', 'Բըրնլի', 'Burnley'],'Լիդս': ['Լիդս', 'Լիդս Յուն', 'Լիդս Յունայթեդ', 'Leeds'],'Սաութհեմփթոն': ['Սաութհեմփթոն', 'Southampton'],'Ֆուլհեմ': ['Ֆուլհեմ', 'Fulham'],'Բրայթոն': ['Բրայթոն', 'Բրայթոն էնդ Հոուվ Ալբիոն', 'Brighton and Hove Albion'],'Վեսթ Հեմ': ['Վեսթ Հեմ', 'Վեստ Հեմ Յունայթեդ', 'West Ham United FC'],'Կրիստալ Փելես': ['Քրիստալ Փելաս', 'Կրիստալ Փելես', 'Քրիսթալ Փելաս', 'Crystal Palace'],'Աստոն Վիլլա': ['Աստոն Վիլլա', 'Aston Villa'],'Տոտենհեմ': ['Տոտենհեմ', 'Տոտենհեմ Հոթսպուր', 'Tottenham'],'Վուլվերհեմփթոն': ['Վուլվերհեմփթոն','Վուլվերհեմպտոն', 'Վուլվերհեմպտոն Ուոնդերերս', 'Wolverhampton'],'Վեսթ Բրոմվիչ': ['Վեսթ Բրոմվիչ Ալբիոն', 'West Bromwich Albion', 'Վեսթ Բրոմվիչ', 'West Bromwich Albion FC'],'Լիվերպուլ': ['Լիվերպուլ', 'Liverpool'],'Էվերթոն': ['Էվերթոն', 'Everton'], 'Շեֆիլդ Յունայթեդ': ['Շեֆֆիլդ Յուն', 'Sheffield United', 'Շեֆիլդ Յունայթեդ'], 'Մանչեսթեր Յունայթեդ': ['Մանչեսթեր Յունայթեդ', 'Manchester United'], 'Մանչեսթեր Սիթի': ['Մանչեսթեր Սիթի', 'Manchester City'], 'Չելսի': ['Չելսի', 'Chelsea'], 'Լեսթեր': ['Լեսթեր', 'Leicester'], 'Նյուքասլ': ['Նյուքասլ Յունայթեդ', 'Newcastle United', 'Նյուքասլ'], 'Արսենալ': ['Արսենալ', 'Arsenal']}
        clubnames_dic = {'Թուրքիա' : ['Թուրքիա', 'Turkey'], 'Իտալիա': ['Իտալիա', 'Italy'], 'Ուելս':['Ուելս', 'Wales'], 'Շվեյցարիա': ['Շվեյցարիա','Switzerland'], 'Դանիա': ['Դանիա', 'Denmark'], 'Ֆինլանդիա':['Ֆինլանդիա','Finland'], 'Բելգիա':['Բելգիա','Belgium'], 'Ռուսաստան':['Ռուսաստան','Russia'], 'Անգլիա':['Անգլիա','England'], 'Խորվաթիա':['Խորվաթիա','Croatia'], 'Ավստրիա':['Ավստրիա','Austria'], 'Հյուսիսային Մակեդոնիա':['Հյուսիսային Մակեդոնիա','Հս Մակեդոնիա', 'North Macedonia'], 'Նիդեռլանդներ':['Նիդեռլանդներ','Netherlands'], 'Ուկրաինա':['Ուկրաինա','Ukraine'], 'Շոտլանդիա':['Շոտլանդիա','Scotland'], 'Չեխիա':['Չեխիա','Czech Republic'], 'Լեհաստան':['Լեհաստան','Poland'], 'Սլովակիա':['Սլովակիա','Slovakia'], 'Իսպանիա':['Իսպանիա','Spain'], 'Շվեդիա':['Շվեդիա','Sweden'], 'Հունգարիա':['Հունգարիա','Hungary'], 'Պորտուգալիա':['Պորտուգալիա','Portugal'], 'Ֆրանսիա':['Ֆրանսիա','France'], 'Գերմանիա':['Գերմանիա','Germany']}

        for key, val in clubnames_dic.items():

            if element in val:

                return key

        return element

    def get_rename_df(self):

        """
        uses function rename to rename all the titles of games of all bookmakers
        """

        for key, dataframe in self.bm_dict.items():

            dataframe['home_team'] = dataframe['away_team'] = ''

            dataframe[['home_team','away_team']] = dataframe['title'].str.split(' vs ', expand = True).values 

            dataframe['home_team'] = dataframe['home_team'].apply(self.rename)

            dataframe['away_team'] = dataframe['away_team'].apply(self.rename)

            dataframe['title'] = dataframe['home_team'] + ' vs ' + dataframe['away_team']

    def sorting(self):

        """
        Sorts all the bookmakers' dataframes

        """

        for key in self.bm_dict.keys():

            self.bm_dict[key].sort_values(by=['date','time','title'], inplace=True)
            self.bm_dict[key].reset_index(drop=True, inplace = True)     


    def get_highest_coef(self):
        """
        Compares all the bookmakers' coefficients and stores everyhing in a dataframe self.highest_coef

        """

        for i in range(len(self.bm_dict['adjarabet'])):


            self.highest_coef.loc[i, 'date'] = self.bm_dict['adjarabet'].loc[i, 'date']
            self.highest_coef.loc[i, 'day'] = self.bm_dict['adjarabet'].loc[i, 'day']
            self.highest_coef.loc[i, 'time'] = self.bm_dict['adjarabet'].loc[i, 'time']
            self.highest_coef.loc[i, 'title'] = self.bm_dict['adjarabet'].loc[i, 'title']

            self.highest_coef.loc[i, 'bm-1'], self.highest_coef.loc[i, '1'] = max({key: self.bm_dict[key].loc[i, '1'] for key in self.bm_dict.keys()}.items(), key=lambda k: k[1])
            self.highest_coef.loc[i, 'bm-X'], self.highest_coef.loc[i, 'X'] = max({key: self.bm_dict[key].loc[i, 'X'] for key in self.bm_dict.keys()}.items(), key=lambda k: k[1])
            self.highest_coef.loc[i, 'bm-2'], self.highest_coef.loc[i, '2'] = max({key: self.bm_dict[key].loc[i, '2'] for key in self.bm_dict.keys()}.items(), key=lambda k: k[1])
            self.highest_coef.loc[i, 'ImplProb'] = (1/self.highest_coef.loc[i, '1'] + 1/self.highest_coef.loc[i, 'X'] + 1/self.highest_coef.loc[i, '2'])
            self.highest_coef.loc[i, 'Margin'] = 1 - self.highest_coef.loc[i, 'ImplProb']
            if self.highest_coef.loc[i, 'Margin'] < 0:
                self.highest_coef.loc[i, 'ShouldBet'] = 0
            else:
                self.highest_coef.loc[i, 'ShouldBet'] = 1
            if self.highest_coef.loc[i, 'ShouldBet'] == 1:

                self.highest_coef.loc[i, 'qt-1'] = (1/self.highest_coef.loc[i,'1']/self.highest_coef.loc[i,'ImplProb'])*1000
                self.highest_coef.loc[i, 'qt-2'] = (1/self.highest_coef.loc[i,'2']/self.highest_coef.loc[i,'ImplProb'])*1000
                self.highest_coef.loc[i, 'qt-X'] = (1/self.highest_coef.loc[i,'X']/self.highest_coef.loc[i,'ImplProb'])*1000
            else:
                self.highest_coef.loc[i, 'qt-1'] = 0
                self.highest_coef.loc[i, 'qt-X'] = 0
                self.highest_coef.loc[i, 'qt-2'] = 0


        print(self.highest_coef)

In [16]:
calculations = Calculations(adjarabet = adjarabet.df, vivaro = vivaro.df, totogaming = totogaming.df, goodwin = goodwin.df)
calculations.get_rename_df()
calculations.sorting()
calculations.get_highest_coef()    

         date           day     time                                  title  \
0    13 Հուն        Կիրակի    20:00        Ավստրիա vs Հյուսիսային Մակեդոնիա   
1    13 Հուն        Կիրակի    23:00                Նիդեռլանդներ vs Ուկրաինա   
2    14 Հուն    Երկուշաբթի    17:00                      Շոտլանդիա vs Չեխիա   
3    14 Հուն    Երկուշաբթի    20:00                    Լեհաստան vs Սլովակիա   
4    14 Հուն    Երկուշաբթի    23:00                       Իսպանիա vs Շվեդիա   
5    15 Հուն     Երեքշաբթի    20:00                Հունգարիա vs Պորտուգալիա   
6    15 Հուն     Երեքշաբթի    23:00                     Ֆրանսիա vs Գերմանիա   
7    16 Հուն    Չորեքշաբթի    17:00                  Ֆինլանդիա vs Ռուսաստան   
8    16 Հուն    Չորեքշաբթի    20:00                        Թուրքիա vs Ուելս   
9    16 Հուն    Չորեքշաբթի    23:00                     Իտալիա vs Շվեյցարիա   
10   17 Հուն     Հինգշաբթի    17:00       Ուկրաինա vs Հյուսիսային Մակեդոնիա   
11   17 Հուն     Հինգշաբթի    20:00                 